## DICOM to Image

This notebook demonstrates how to convert a Digital Imaging and Communications in Medicine (DICOM) file to one or more images in PNG or JPEG format.

This example uses:
- `pydicom` library for reading and parsing DICOM files.
- `cv2` (installed as `opencv-python`) for image processing, color space conversions, and saving as JPG/PNG.
- `numpy` for numerical operations and handling arrays, useful to manipulate image pixel data.

In [ ]:
# Imports
from pydicom.pixel_data_handlers.util import (
    apply_voi_lut,
    convert_color_space,
    apply_modality_lut,
)
from pathlib import Path

import numpy as np
import pydicom
import cv2

In [ ]:
# DICOM source file (`.dcm` extension).
SRC_PATH = "./example/input/sample_ybr_full_422_2.dcm"

# Output path to store each frame.
OUT_PATH = Path("./example/output/")

# Output image type (can be "png" or "jpg").
EXT = "jpg"

### Load DICOM file and its image data

In [ ]:
ds = pydicom.dcmread(SRC_PATH)
pixel_array = ds.pixel_array

### Apply Modality LUT

DICOMs may contain a Modality Look-Up Table (LUT) to transform **raw data** from imaging devices into standardized units (i.e. Hounsfield units which quantify tissue density) also known as "modality-specific values" to ensure consistency regardless of the device used to acquire the image.

In [ ]:
if "ModalityLutSequence" in ds or ("RescaleSlope" in ds and "RescaleIntercept" in ds):
    pixel_array = apply_modality_lut(pixel_array, ds)
else:
    rescale_slope = ds.get("RescaleSlope") or 1.0
    rescale_intercept = ds.get("RescaleIntercept") or 0.0
    pixel_array = pixel_array * rescale_slope + rescale_intercept

### Apply VOI LUT or Window Level

DICOMs may contain a VOI LUT (Value of Interest Look-Up Table) to transform modality-specific values into pixel values that best reflect the desired content or area of interest. For example, to enhance specific features in medical images to highlighting tumors or blood vessels.

In [ ]:
pixel_array = pixel_array.astype("int")
pixel_array = apply_voi_lut(pixel_array, ds)

### Convert source image type to RGB color space

The DICOM file may contain an image that is grayscale (MONOCHROME1 or MONOCHROME2), RGB, YBR, or palette color. The code below converts the source image from its respective color space to RGB color space, which is a requirement for creating a PNG or JPG file.

- The "PALETTE COLOR" type is not handled, as it is more complicated and out of the scope of this example notebook.
- The "MONOCHROME2" type does not need conversion because pixel values are already dark-to-bright in ascending order.
- And the "RGB" type is already in the target color space.

In [ ]:
# Handle each source image type.
image_type = ds.get("PhotometricInterpretation")

if image_type == "MONOCHROME1":
    pixel_array = np.amax(pixel_array) - pixel_array
elif image_type in ("RGB", "YBR_FULL", "YBR_FULL_422"):
    pixel_array = convert_color_space(pixel_array, image_type, "RGB").astype("float32")
    pixel_array = cv2.cvtColor(pixel_array, cv2.COLOR_RGB2BGR)

# Normalize the pixel array to values ranging from 0 to 255 for JPG/PNG.
pixel_array = cv2.normalize(pixel_array, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

### Save DICOM frames to the output folder

In [ ]:
if ds.get("NumberOfFrames", 1) > 1:
    for idx, frame in enumerate(pixel_array):
        cv2.imwrite(str(OUT_PATH / f"frame_{idx}.{EXT}"), frame)
else:
    cv2.imwrite(str(OUT_PATH / f"frame.{EXT}"), pixel_array)